<a href="https://colab.research.google.com/github/stef4k/train-maintenance-data-mining/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text classification


In [3]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from collections import Counter
import ast


from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, RandomOverSampler
from imblearn.combine import SMOTEENN, SMOTETomek
from gensim.models import Word2Vec
from scipy.optimize import minimize
from sklearn.base import TransformerMixin
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, RandomOverSampler
from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from scipy.sparse import csr_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from xgboost import XGBClassifier
from tqdm import tqdm
from scipy.optimize import differential_evolution
import numpy as np
import pandas as pd
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")


Manually remove the first ';' from the first row in csv file

In [4]:
df = pd.read_csv('sncb_data_challenge.csv', delimiter=';', index_col=0)
df.sample(2)

,incident_id,vehicles_sequence,events_sequence,seconds_to_incident_sequence,approx_lat,approx_lon,train_kph_sequence,dj_ac_state_sequence,dj_dc_state_sequence,incident_type
17,4433927,"[667, 667, 667, 667, 667, 667, 667, 667, 667, ...","[2708, 2744, 4026, 4020, 2852, 3634, 2854, 406...","[-14232, -14232, -14232, -14231, -14201, -1420...",50.897397,4.399000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...",9
326,4449091,"[574, 574, 574, 574, 574, 574, 574, 574, 574, ...","[4004, 4032, 4028, 2852, 4026, 4110, 2854, 274...","[-4468, -4468, -4467, -4466, -4466, -4466, -44...",51.160264,4.158203,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...",2


In [5]:
class Word2VecVectorizer(TransformerMixin):
    def __init__(self, size=100, window=5, min_count=1, workers=4):
        self.size = size
        self.window = window
        self.min_count = min_count
        self.workers = workers
        self.w2v_model = None

    def fit(self, X, y=None):
        sentences = [sentence.split() for sentence in X]
        self.w2v_model = Word2Vec(sentences, vector_size=self.size, window=self.window,
                                  min_count=self.min_count, workers=self.workers)
        return self

    def transform(self, X, y=None):
        transformed_data = np.array([
            np.mean([self.w2v_model.wv[word] for word in sentence.split() if word in self.w2v_model.wv]
                    or [np.zeros(self.
                                 size)], axis=0)
            for sentence in X
        ])
        return csr_matrix(transformed_data)

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [6]:

class EnsembleModel:
    def __init__(self, trained_models, trained_vectorizers):

        self.trained_models = trained_models
        self.trained_vectorizers = trained_vectorizers
        self.optimized_weights = None

    def _generate_prediction_matrix(self, X):
        predictions = {}
        for (vect_name, samp_name, clf_name), model in self.trained_models.items():
            vectorizer = deepcopy(self.trained_vectorizers[vect_name])
            X_vect = vectorizer.transform(X).toarray()

            # Collect probability predictions for multiclass
            predictions[(vect_name, samp_name, clf_name)] = model.predict_proba(X_vect)

        # Convert predictions to a 3D array: (num_samples, num_classes, num_models)
        return np.stack(list(predictions.values()), axis=2)

    def fit(self, X_train, y_train):
        pred_matrix = self._generate_prediction_matrix(X_train)

        # Define fitness function for genetic algorithm
        def fitness(weights):
            # Apply weights to model predictions
            weighted_pred = np.tensordot(pred_matrix, weights, axes=([2], [0]))
            # Get the predicted class with the highest weighted probability
            final_pred = np.argmax(weighted_pred, axis=1)
            # Return negative weighted F1 score for optimization
            return -f1_score(y_train, final_pred, average='weighted')

        # Genetic algorithm for weight optimization
        num_models = pred_matrix.shape[2]
        bounds = [(0, 1)] * num_models
        result = differential_evolution(fitness, bounds)

        # Store optimized weights
        self.optimized_weights = result.x

    def predict(self, X):
        if self.optimized_weights is None:
            raise ValueError("The ensemble model must be trained using `train_ensemble` before prediction.")

        # Generate predictions for the input data
        pred_matrix = self._generate_prediction_matrix(X)
        weighted_pred = np.tensordot(pred_matrix, self.optimized_weights, axes=([2], [0]))
        ensemble_pred = np.argmax(weighted_pred, axis=1)
        return ensemble_pred


In [7]:

class Representations:
  def __init__(self, df):
    self.df = df

  def representation_a(self, df):
            events_types_dict = {}
            for events_sequence in df['events_sequence']:
                row_list = ast.literal_eval(events_sequence)  # Transforming string into actual list
                unique_events = set(row_list)
                for event in unique_events:
                    if not events_types_dict.get(event):
                        events_types_dict[event] = 0
                    events_types_dict[event] += 1

            # Step 2: Sort events by frequency and calculate percentages
            sorted_dict = dict(sorted(events_types_dict.items(), key=lambda item: item[1], reverse=True))
            sorted_events_perc_df = pd.DataFrame(list(sorted_dict.items()), columns=['event_type', 'frequency'])
            sorted_events_perc_df['percentage'] = sorted_events_perc_df['frequency'] / df.shape[0] * 100
            sorted_events_perc_df['event_type'] = sorted_events_perc_df['event_type'].astype(str)  # Ensure event_type is string

            # Step 3: Filter events with percentage <= 85%
            events_low_frequency = list(map(int, list(sorted_events_perc_df[sorted_events_perc_df.percentage <= 85].event_type)))

            # Step 4: Clean the 'events_sequence' column
            df['clean_events_sequence'] = (
                df['events_sequence']
                .apply(ast.literal_eval)  # Convert string to list
                .apply(lambda x: [i for i in x if i in events_low_frequency])  # Filter low-frequency events
                .astype(str)
                .replace(r'[\[\],]', '', regex=True)  # Remove brackets and commas
            )

            # Step 5: Extract the target column
            self.y = df['incident_type'].copy()  # Separate target column
            self.X = df['clean_events_sequence'].copy()
            return self.X, self.y

  def representation_b(self, df):

    before_incident = []
    after_incident = []

    for _, row in df.iterrows():
        # Convertir las cadenas a listas
        events = ast.literal_eval(row['events_sequence'])
        seconds = ast.literal_eval(row['seconds_to_incident_sequence'])
        incident_type = row['incident_type']

        # Filtrar eventos antes del incidente (t <= 0)
        before_events = " ".join([str(event) for event, time in zip(events, seconds) if time <= 0])
        if before_events:
            before_incident.append({
                "events_sequence": before_events,
                "class": incident_type
            })

        # Filtrar eventos después del incidente (t > 0)
        after_events = " ".join([str(event) for event, time in zip(events, seconds) if time > 0])
        if after_events:
            after_incident.append({
                "events_sequence": after_events,
                "class": 100
            })

    # Convertir listas a DataFrames
    before_df = pd.DataFrame(before_incident)
    after_df = pd.DataFrame(after_incident)

    return before_df, after_df


  def representation_c(self, df, sequence_length=30):

    overlapping_sequences = []
    step = sequence_length // 2  # Paso entre ventanas solapadas

    for _, row in df.iterrows():
        # Convertir eventos y tiempos en listas
        events = ast.literal_eval(row['events_sequence'])
        seconds = ast.literal_eval(row['seconds_to_incident_sequence'])
        incident_type = row['incident_type']

        # Generar cadenas antes del incidente
        for i in range(0, len(events) - sequence_length + 1, step):
            sequence = [str(i) for i in events[i:i + sequence_length]]
            seconds_slice = seconds[i:i + sequence_length]
            sequence_class = incident_type

            overlapping_sequences.append({"sequence": " ".join(sequence), "class": sequence_class})

    # Crear un nuevo DataFrame con las secuencias generadas
    sequences_df = pd.DataFrame(overlapping_sequences)
    return sequences_df.sequence, sequences_df['class']


In [8]:

class Experiment:
    def __init__(self, X, y):
        self.X = X
        self.le = LabelEncoder()
        self.y = self.le.fit_transform(y)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)
        self.y_train = self.le.transform(self.y_train)
        self.y_test = self.le.transform(self.y_test)
        self.trained_models = {}
        self.trained_vectorizers = {}
        self.trainer_samplers = {}
        self.results = []
        self.sampling_strategies = {
            "SMOTE": SMOTE(sampling_strategy='auto', random_state=1, k_neighbors=3),
            #"Borderline-SMOTE": BorderlineSMOTE(sampling_strategy='auto', random_state=1, k_neighbors=3),
            #"ADASYN": ADASYN(sampling_strategy='auto', random_state=1, n_neighbors=3),
            #"RandomOversampler": RandomOverSampler(sampling_strategy='auto', random_state=1),
            #"SMOTE-ENN": SMOTEENN(sampling_strategy='auto', random_state=1),
            #"SMOTE-Tomek": SMOTETomek(sampling_strategy='auto', random_state=1)
        }

        self.vectorizers = {
            #"TFIDF": TfidfVectorizer(),
            "Count": CountVectorizer(),
            #"Word2Vec": Word2VecVectorizer(size=100, window=5, min_count=1)
        }
        self.classifiers = {
            'LogisticRegression': LogisticRegression(),
            'DecisionTree': DecisionTreeClassifier(),
            'RandomForest': RandomForestClassifier(),
            'ExtraTreesClassifier': ExtraTreesClassifier(),
            'GradientBoostingClassifier': GradientBoostingClassifier(),
            'AdaBoostClassifier': AdaBoostClassifier(),
            'GaussianNB': GaussianNB(),
            'KNN': KNeighborsClassifier(),
            'SVM': SVC(probability=True),
            'XGBoost': XGBClassifier(objective="multi:softmax", num_class=len(np.unique(y)), eval_metric="mlogloss", use_label_encoder=False),
        }


    def duplicate_minor_classes(self, X, y, min_instances=5):
        X = X.reset_index(drop=True)
        y  = y.reset_index(drop=True)
        class_counts = y.value_counts()

        # Identificar las clases minoritarias
        minor_classes = class_counts[class_counts < min_instances].index

        # Filtrar las filas correspondientes a las clases minoritarias
        minor_class_rows = X.loc[y.isin(minor_classes)]
        minor_class_labels = y.loc[y.isin(minor_classes)]

        # Duplicar las filas y etiquetas correspondientes
        duplicated_X = pd.concat([minor_class_rows] * 2, ignore_index=True)
        duplicated_y = pd.concat([minor_class_labels] * 2, ignore_index=True)

        # Combinar con las filas y etiquetas originales
        X_balanced = pd.concat([X, duplicated_X], ignore_index=True)
        y_balanced = pd.concat([y, duplicated_y], ignore_index=True)

        X_balanced.reset_index(drop=True, inplace=True)
        y_balanced.reset_index(drop=True, inplace=True)

        return X_balanced, y_balanced


    def test(self, model, model_name, vectorizer, vectorizer_name, sampler, sampler_name):
        """Test a model with Stratified K-Fold and return an array with metric results."""
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        accuracies, recalls, precisions, f1s = [], [], [], []
        X, y = self.X_train, self.y_train

        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            X_train = vectorizer.fit_transform(X_train).toarray()
            X_test = vectorizer.transform(X_test).toarray()

            X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)

            model.fit(X_resampled, y_resampled)

            y_pred = model.predict(X_test)

            # Collect metrics for each fold
            accuracies.append(accuracy_score(y_test, y_pred))
            recalls.append(recall_score(y_test, y_pred, average='weighted'))
            precisions.append(precision_score(y_test, y_pred, average='weighted'))
            f1s.append(f1_score(y_test, y_pred, average='weighted'))

        # Return average metrics as a result array
        return [
            model_name, vectorizer_name, sampler_name,
            np.mean(accuracies), np.std(accuracies),
            np.mean(recalls), np.std(recalls),
            np.mean(precisions), np.std(precisions),
            np.mean(f1s), np.std(f1s)
        ]

    def training(self):
        results = []
        progress_bar = tqdm(total = len(self.vectorizers.keys()) * len(self.sampling_strategies.keys()) * len(self.classifiers.keys()))
        self.X_train, self.y_train = self.duplicate_minor_classes(self.X_train, pd.Series(self.y_train))
        # Iterate over vectorizers, samplers, and classifiers
        for vect_name, ovectorizer in self.vectorizers.items():
            vectorizer = deepcopy(ovectorizer)
            for samp_name, osampler in self.sampling_strategies.items():
                sampler = deepcopy(osampler)
                for clf_name, omodel in self.classifiers.items():
                    model = deepcopy(omodel)

                    result = self.test(
                        model=model,
                        model_name=clf_name,
                        vectorizer=vectorizer,
                        vectorizer_name=vect_name,
                        sampler=sampler,
                        sampler_name=samp_name
                    )
                    # Store the trained model and add result to the results list
                    results.append(result)

                    vectorizer = deepcopy(ovectorizer)
                    sampler = deepcopy(osampler)
                    model = deepcopy(omodel)
                    vectorizer.fit(self.X_train)
                    X_train = vectorizer.transform(self.X_train).toarray()
                    X_resampled, y_resampled = sampler.fit_resample(X_train, self.y_train)
                    model.fit(X_resampled, y_resampled)
                    self.trained_models[(vect_name, samp_name, clf_name)] = model
                    self.trained_vectorizers[vect_name] = vectorizer
                    self.trainer_samplers[(vect_name, samp_name)] = sampler
                    progress_bar.update(1)

        # Convert results to a DataFrame for better readability
        columns = [
            'Model', 'Vectorizer', 'Sampler',
            'Accuracy Mean', 'Accuracy Std',
            'Recall Mean', 'Recall Std',
            'Precision Mean', 'Precision Std',
            'F1 Mean', 'F1 Std'
        ]
        results_df = pd.DataFrame(results, columns=columns)
        self.results = results_df
        return results_df




In [9]:
X, y = df.events_sequence, df.incident_type
exp = Experiment(X, y)
results = exp.training()
ensemble = EnsembleModel(exp.trained_models, exp.trained_vectorizers)
ensemble.fit(exp.X_train, exp.y_train)
ensemble_predictions = ensemble.predict(exp.X_test)
print(classification_report(exp.y_test, ensemble_predictions, zero_division=1))

  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
results

In [ ]:
X, y = Representations(df).representation_a(df)
exp = Experiment(X, y)
results = exp.training()
ensemble = EnsembleModel(exp.trained_models, exp.trained_vectorizers)
ensemble.fit(exp.X_train, exp.y_train)
ensemble_predictions = ensemble.predict(exp.X_test)
print(classification_report(exp.y_test, ensemble_predictions, zero_division=1))

In [ ]:
results

In [ ]:
df_before, df_after = Representations(df).representation_b(df)
X, y = pd.concat([df_before.events_sequence, df_after.events_sequence]), pd.concat([df_before['class'], df_after['class']])
exp = Experiment(X, y)
results = exp.training()
ensemble = EnsembleModel(exp.trained_models, exp.trained_vectorizers)
ensemble.fit(exp.X_train, exp.y_train)
ensemble_predictions = ensemble.predict(exp.X_test)
print(classification_report(exp.y_test, ensemble_predictions, zero_division=1))

In [ ]:
results

In [ ]:
X, y = Representations(df).representation_c(df, sequence_length=100)
exp = Experiment(X, y)
results = exp.training()
ensemble = EnsembleModel(exp.trained_models, exp.trained_vectorizers)
ensemble.fit(exp.X_train, exp.y_train)
ensemble_predictions = ensemble.predict(exp.X_test)
print(classification_report(exp.y_test, ensemble_predictions, zero_division=1))


 80%|████████  | 8/10 [1:49:50<23:31, 705.86s/it] 

In [ ]:
results